In [1]:
from collections import defaultdict, ChainMap
import itertools as it
import re
from string import Template

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
html_files = [
    {'week': 9, 'collected': '2021-06-25'},
    {'week': 4, 'collected': '2021-05-20'},
    {'week': 2, 'collected': '2021-04-26'},
    {'week': 1, 'collected': '2021-04-19'},
]

In [3]:
score_key_template = Template('week_${week}_score')

def get_current_player_scores(soup, week):
    current_player_score_rows = soup.find_all('tr', player_id=True, player_role='overall')
    score_key = score_key_template.substitute(week=week)
    return (
        {
            'player_id': row['player_id'],
            'player_name': row.find(class_='ibm-player-name').contents[0],
            score_key: int(row.find(class_='ibm-rating').contents[0]),
        } for row in current_player_score_rows
    )

score_history_regex = re.compile('^player_impact_Chart_(?P<player_id>\d+)_overall$')

def transform_to_score(scorestr):
    try:
        return int(scorestr)
    except ValueError:
        return None


def get_player_score_history(canvas):
    score_history = {
        score_key_template.substitute(week=index + 1): transform_to_score(scorestr)
        for index, scorestr in enumerate(canvas['impactdata'].split(','))
    }

    return {'player_id': re.match(score_history_regex, canvas['id']).group('player_id')} | score_history

def get_player_score_histories(soup):
    return (get_player_score_history(canvas) for canvas in soup.find_all('canvas', id=score_history_regex))

def consolidate_score_snapshots(score_snapshots):
    player_scores = defaultdict(ChainMap)
    for score_snapshot in score_snapshots:
        player_id = score_snapshot.pop('player_id')
        player_scores[player_id].maps.append(score_snapshot)
    return [dict(chained_score_data) for chained_score_data in player_scores.values()]

In [4]:
html_file_template = Template('raw-html/${collected}.html')

def get_score_snapshots_from_file(week, collected):
    html_file = html_file_template.substitute(collected=collected)
    with open(html_file) as f:
        soup = BeautifulSoup(f)

    yield from get_current_player_scores(soup, week)
    yield from get_player_score_histories(soup)

def get_complete_score_history():
    consolidated_scores = consolidate_score_snapshots(
        it.chain(*[
            get_score_snapshots_from_file(**file_metadata)
            # The order shouldn't matter, but we prioritize the most recently-collected data
            # in case there were score presentation errors that were corrected
            for file_metadata in sorted(html_files, key=lambda m: m['week'], reverse=True)
        ])
    )
    return pd.DataFrame.from_records(consolidated_scores).set_index('player_name').astype(pd.UInt32Dtype())

In [5]:
scores = get_complete_score_history()

In [6]:
scores[scores.isna().sum(axis='columns') > 1]

,week_1_score,week_2_score,week_3_score,week_4_score,week_5_score,week_6_score,week_7_score,week_8_score,week_9_score
player_name,,,,,,,,,
LIP,107,<NA>,130,135,<NA>,136,139,121,124
Decay,<NA>,124,129,122,<NA>,116,113,120,120
shy,<NA>,<NA>,94,<NA>,<NA>,110,108,114,114
MAG,<NA>,139,134,128,<NA>,115,111,112,112
smurf,100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,110,109
Assassin,<NA>,135,130,123,<NA>,106,102,109,109
moth,69,117,102,<NA>,<NA>,<NA>,<NA>,110,109
Guxue,<NA>,82,92,94,<NA>,108,106,108,108
MCD,<NA>,70,88,89,<NA>,103,101,108,108


In [7]:
scores.isna().sum(axis='rows')

week_1_score     30
week_2_score      4
week_3_score      2
week_4_score      4
week_5_score    107
week_6_score     11
week_7_score      9
week_8_score      7
week_9_score      7
dtype: int64

In [8]:
scores.to_csv('ibm_power_scores.csv')